In [ ]:
import torch
# torch.zeros(1).cuda()
torch.cuda.is_available()

In [ ]:
from dataset import get_train_val_locs

train_locs, val_locs = get_train_val_locs(n_val_locs=10000)

In [ ]:
from dataset import load_evo_dataset
evo_ds = load_evo_dataset()
v3_locs = evo_ds.locs

In [ ]:
v3_locs.columns

In [ ]:
from evotrain.v2 import EvoTrainV2Dataset

In [ ]:
from dataset import EvoNetV1Dataset

In [ ]:
ds = EvoNetV1Dataset(bands=[
                "B02",
                "B03",
                "B04",
                "B08",
                "B05",
                "B06",
                "B07",
                "B8A",
                "B11",
                "B12",
                "cop-DEM-alt"])

In [ ]:
list(ds.ids).index('55GEN_046_09_20200105')

In [ ]:
feats, feats_head, probs, weights = ds[121312]

In [ ]:
feats.shape, feats_head.shape

In [ ]:
import json

with open("/projects/TAP/vegteam/models_wdk/lsc10/v15/lsc10-v15_test/config.json", "r") as f:
    config = json.load(f)

In [ ]:
from model import Net

In [ ]:
net = Net(**config["model"])

In [ ]:
import numpy as np
x, x_head, y, y_weights = ds[121318]
x = torch.from_numpy(x.numpy()[np.newaxis, ...])
x_head = torch.from_numpy(x_head.numpy()[np.newaxis, ...])
y_pred = net.evonet(x, x_head)

In [ ]:
y_pred = [net._slice_cover(y_pred), net._slice_occlusion(y_pred), net._slice_ecosystems(y_pred)]

In [ ]:
y = [torch.from_numpy(it.numpy()[np.newaxis, ...]) for it in y]

In [ ]:
y_weights = [torch.from_numpy(it.numpy()[np.newaxis, ...]) for it in y_weights]

In [ ]:
print(y_pred[0].shape)
print(y[0].shape)
print(y_weights[0].shape)

In [ ]:
net.on_validation_start()
net.num_classes_components['occlusion']
net._iou_accumulators[labels_type]

In [ ]:
from torch import nn
softmax = nn.Softmax(dim=1)
y_pred_cover = softmax(y_pred[0])
y_pred_occlusion = softmax(y_pred[1])
y_pred_ecosystems = softmax(y_pred[2])
y_pred_softmax = [y_pred_cover, y_pred_occlusion, y_pred_ecosystems]
net._update_iou_accumulators(y_pred_softmax, y, y_weights)

In [ ]:
labels_type = 'occlusion'
net._iou_accumulators[labels_type]

In [ ]:
mean_iou, classes_iou = net._get_iou_metric(labels_type)
classes_dict = net.classes_dict[labels_type]
classes_ids = list(classes_dict.keys())
classes_ids

In [ ]:
classes_iou

In [ ]:
classes_dict = net.classes_dict[labels_type]

In [ ]:
prob_cover.shape, prob_occlusion.shape, prob_eco.shape, weight_cover.shape, weight_occlusion.shape, weight_eco.shape

In [ ]:
len(v3_locs)

In [ ]:
len(list(set(v3_locs['location_id'])))

In [ ]:
v3_locs.columns

In [ ]:
list(set(v3_locs['selection_1']))

In [ ]:
train_locs[0]

In [ ]:
len(train_locs)

In [ ]:
len(val_locs)

In [ ]:
import pandas as pd
'/projects/TAP/vegteam/training_data/evotrain_v2/metadata/meteo_biome_features_v1.parquet'
tst_meteo = pd.read_parquet('/home/vito/dekeersw/repos/evotrain/src/evotrain/v2/metadata/meteo_biome_features_v1.parquet')

In [ ]:
tst_meteo


In [ ]:
# feats_head
# feats

In [ ]:
# config["model"]

In [ ]:
# y_pred

In [ ]:

labels_type = 'occlusion'
range(net.num_classes_components[labels_type])

In [ ]:
net._iou_accumulators[labels_type]

In [ ]:
net._labels_types
label_type = 'occlusion'
mean_iou, classes_iou = net._get_iou_metric(label_type)

In [ ]:
feats.unsqueeze(0).shape

In [ ]:
y = net(feats.unsqueeze(0), feats_head.unsqueeze(0))

In [ ]:
y = net(feats.unsqueeze(0), feats_head.unsqueeze(0))

In [ ]:
len(y)

In [ ]:
y[0].shape

In [ ]:
y[1].shape

In [ ]:
y[2].shape

In [ ]:
y[0].shape

In [ ]:
weights[0].shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from labels import binarize_probs
from labels import label_to_rgb
from labels import LSC_COVER_LABELS
nodata_mask = (weights[0][0] < 0.1).squeeze()
lsc_probs = probs[0]
# lsc_probs[:, nodata_mask] = np.nan
lsc_map = binarize_probs(lsc_probs, LSC_COVER_LABELS) 
lsc_map[nodata_mask] = 0
rgb = label_to_rgb(lsc_map, 'cover')

print(rgb.shape)
plt.imshow(np.moveaxis(rgb,[0,1,2],[2,0,1]))

In [ ]:
import numpy as np
import xarray as xr
prob_cover = ds.evo_dataset.read_annotation(
            sample_id, annotation_name="lsc-evo-v1-prob-cover-v1"
        )
print(len(prob_cover.band))
sample_id = '55GEN_046_09_20200105'
tst = ds.evo_dataset.read_annotation(
            sample_id, annotation_name="lsc-evo-v1-weight-cover-v1"
        )
xr.concat([tst for it in np.arange(len(prob_cover.band))], dim='band')
# xr.DataArray(np.repeat(tst.data, 4, axis=0), dims=['band', 'y', 'x'],
#             coords=)


In [ ]:
from pathlib import Path
import pandas as pd
_DATASET_PATH_HPC = Path("/projects/TAP/vegteam/training_data/evotrain_lsc/evotrain_lsc_v4_iiasa")
meta = pd.read_parquet(_DATASET_PATH_HPC / 'metadata' / "locs_fracs.parquet")

In [ ]:
len(list(set(meta['location_id'])))